In [1]:
albums_audio_path = './audio-cds/'
albums_chords = './chordlab/'

import glob
import os

list_dir = [x[0] for x in os.walk(albums_audio_path)]
audio_files = []

for folder in list_dir:
    list_files = glob.glob(folder + '/*.flac')
    for file in list_files:
        audio_files.append(file)

list_dir = [x[0] for x in os.walk(albums_chords)]
chord_files = []

for folder in list_dir:
    list_files = glob.glob(folder + '/*.tsv')
    for file in list_files:
        chord_files.append(file)

audio_and_chord = {}
for i, audio_file in enumerate(audio_files):
    audio_and_chord[audio_file] = chord_files[i]
    
audio_and_chord

{'./audio-cds/01_-_Please_Please_Me\\01_-_I_Saw_Her_Standing_There.flac': './chordlab/01_-_Please_Please_Me\\01_-_I_Saw_Her_Standing_There.lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\02_-_Misery.flac': './chordlab/01_-_Please_Please_Me\\02_-_Misery.lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\03_-_Anna_(Go_To_Him).flac': './chordlab/01_-_Please_Please_Me\\03_-_Anna_(Go_To_Him).lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\04_-_Chains.flac': './chordlab/01_-_Please_Please_Me\\04_-_Chains.lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\05_-_Boys.flac': './chordlab/01_-_Please_Please_Me\\05_-_Boys.lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\06_-_Ask_Me_Why.flac': './chordlab/01_-_Please_Please_Me\\06_-_Ask_Me_Why.lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\07_-_Please_Please_Me.flac': './chordlab/01_-_Please_Please_Me\\07_-_Please_Please_Me.lab.pcs.tsv',
 './audio-cds/01_-_Please_Please_Me\\08_-_Love_Me_Do.flac': './chordlab/01_-_Please_Please_Me\\08_-_L

In [2]:
#creating a df of data with labels of all audio files

import tfr
import numpy as np
import pandas as pd
import librosa
import time

start_time = time.time()

def time_to_samples(time, samplerate):
    return np.round(time * samplerate)

def block_labels(df_blocks, df_labels):
    df_labelled_blocks = pd.DataFrame(columns=['start', 'end', 'label'])
    for idx, row in df_frames.iterrows():
        for ix, row_labels in df_labels.iterrows():
            if row['start'] >= row_labels['start'] and row['end'] <= row_labels['end']:
                df_labelled_blocks = df_labelled_blocks.append({'start': row['start'], 
                                                                'end': row['end'], 
                                                                'label': row_labels['label']
                                                               },ignore_index=True)
                break

    return df_labelled_blocks

def create_df_with_preprocessed_signal_and_label(signal_frames, df_labelled_blocks):
    df_data_labelled = pd.DataFrame(columns=['signal', 'label'])
    for idx, row in df_labelled_blocks.iterrows():
        audio_chunk = signal_frames.frames[idx]
        transformed_X = np.abs(np.fft.rfft(audio_chunk)) #Fourier transform on each sample
        df_data_labelled = df_data_labelled.append({'signal': transformed_X, 'label': row['label']}, ignore_index=True)
    return df_data_labelled

frame_size = 4096
hop_size = 4096
samplerate = 44100

i = 0
global_df_data_labelled = pd.DataFrame(columns=['signal', 'label'])
for audio_file_path, chords_file_path in audio_and_chord.items():
    chord_file = pd.read_csv(chords_file_path, sep='\t')
    
    signal_frames = tfr.SignalFrames(audio_file_path, frame_size=frame_size, hop_size=hop_size)
    
    chord_file['start_sample'] = time_to_samples(chord_file['start'], samplerate)
    chord_file['end_sample'] = time_to_samples(chord_file['end'], samplerate)
    df_frames = pd.DataFrame({'start': time_to_samples(signal_frames.start_times, samplerate).astype(np.int64)})
    df_frames['end'] = df_frames['start'] + frame_size
    
    df_labels = chord_file[['start_sample', 'end_sample', 'label']].copy()
    df_labels.rename(columns={'start_sample': 'start', 'end_sample': 'end'}, inplace=True)
    
    df_labelled_blocks = block_labels(df_frames, df_labels)
    
    df_data_labelled = create_df_with_preprocessed_signal_and_label(signal_frames, df_labelled_blocks)
    global_df_data_labelled = global_df_data_labelled.append(df_data_labelled)
    
    
    i+=1
#     if i==3:
#         break

    print("Processed files: %i/%i" %(i, len(audio_and_chord)))
    
global_df_data_labelled.to_pickle('./data/global_df_data_labelled_fft_4096.pkl')

elapsed_time = time.time() - start_time
print("This process took %.2f seconds" %(elapsed_time))

Processed files: 1/180
Processed files: 2/180
Processed files: 3/180
Processed files: 4/180
Processed files: 5/180
Processed files: 6/180
Processed files: 7/180
Processed files: 8/180
Processed files: 9/180
Processed files: 10/180
Processed files: 11/180
Processed files: 12/180
Processed files: 13/180
Processed files: 14/180
Processed files: 15/180
Processed files: 16/180
Processed files: 17/180
Processed files: 18/180
Processed files: 19/180
Processed files: 20/180
Processed files: 21/180
Processed files: 22/180
Processed files: 23/180
Processed files: 24/180
Processed files: 25/180
Processed files: 26/180
Processed files: 27/180
Processed files: 28/180
Processed files: 29/180
Processed files: 30/180
Processed files: 31/180
Processed files: 32/180
Processed files: 33/180
Processed files: 34/180
Processed files: 35/180
Processed files: 36/180
Processed files: 37/180
Processed files: 38/180
Processed files: 39/180
Processed files: 40/180
Processed files: 41/180
Processed files: 42/180
P